In [1]:

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random


In [2]:
# Files
dir = "data/"
image = "image.png"
qmatrix = "qmatrix.png"

# Data
original = Image.open(dir + image, 'r')
original_qmatrix = Image.open(dir + qmatrix, 'r')

# Metadata
width, height = original.size

# Load and Show Picture
image = original

def show_image(image_title):
    plt.imshow(image)
    plt.title(image_title)
    plt.show()
    print(image_title + " : " + str((np.asarray(image)[0, 0])))
    
    
show_image('Original') # Mark 1


Original : [ 1  9 18]


In [3]:
image = np.asarray(original, "int32") / 255
def dodct(r, g, b):
    return [cv2.dct(r),
            cv2.dct(g),
            cv2.dct(b)]

def doidct(r, g, b):
    return [cv2.idct(r),
            cv2.idct(g),
            cv2.idct(b)]

r, g, b = cv2.split(image)

image = dodct(r, g, b)
image = np.reshape(image, [height, width, 3])

qmatrix = np.asarray(original_qmatrix, "int32")
blocksize = 8

def blockfunction(block):
    block[:, :, 0] = np.float32(np.divide(block[:, :, 0], qmatrix[:, :]))
    block[:, :, 1] = np.float32(np.divide(block[:, :, 1], qmatrix[:, :]))
    block[:, :, 2] = np.float32(np.divide(block[:, :, 2], qmatrix[:, :]))
    return block


# Loop every block 8x8
for x in range(0, width, blocksize):
    x_end = x + blocksize if x + blocksize < width else width
    for y in range(0, height, blocksize):
        y_end = y + blocksize if y + blocksize < height else height
        block = image[y:y_end, x:x_end]
        block = blockfunction(block)
        np.clip(block, 0, 255, block)
show_image('test')
image = np.reshape(image, [3, height * width])
r, g, b = image
image = doidct(r, g, b)
image = np.reshape(image, [height, width, 3])
show_image('DCT')


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


test : [3.5347693 0.        0.       ]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


DCT : [0.85723318 0.25843781 0.22251209]
